In [1]:
# Import required libraries
#install dash
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

dropdown_options=[{'label': 'All Sites', 'value': 'ALL'}]
for site in spacex_df["Launch Site"].unique():
    dropdown_options.append({'label': site, 'value': site})
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                dcc.Dropdown(id='site-dropdown',
                                    options=dropdown_options,
                                    value='ALL',
                                    placeholder="Select a Launch Site here",
                                    searchable=True
                                    ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                    min=0, max=10000, step=1000,
                                    marks={0: '0', 1000: '1000', 2000: '2000', 3000: '3000',
                                    4000: '4000', 5000: '5000', 6000: '6000', 7000: '7000',
                                    8000: '8000', 9000: '9000', 10000: '10000'},
                                    value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

# Function decorator to specify function input and output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        filtered_df = spacex_df[spacex_df["class"]==1].groupby("Launch Site", as_index=False).sum()
        fig = px.pie(filtered_df, values=filtered_df["class"], 
        names=filtered_df["Launch Site"],
        title='Total Successful Launches by Launch Site')
    else:
        filtered_df = spacex_df[spacex_df["Launch Site"]==entered_site]
        filtered_df = filtered_df[["class", "Launch Site"]].groupby("class").count().reset_index()
        fig = px.pie(filtered_df, values=filtered_df["Launch Site"], 
        names=(filtered_df["class"].replace({0: 'Failure', 1: 'Success'})),
        title='Success vs Failure for Launches from {}'.format(entered_site))
    return fig
        # return the outcomes piechart for a selected site

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
    Input(component_id="payload-slider", component_property="value")])

def scatter(entered_site, payload_range):
    filtered_df = spacex_df[payload_range[0] <= spacex_df["Payload Mass (kg)"]]
    filtered_df = filtered_df[payload_range[1] >= filtered_df["Payload Mass (kg)"]]
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, y="class", x="Payload Mass (kg)",
        color="Booster Version Category",
        title='Outcome of Launches with Payload Mass between {} and {} kg'.format(payload_range[0], payload_range[1]))
    else:
        filtered_df = filtered_df[filtered_df["Launch Site"]==entered_site]
        fig = px.scatter(filtered_df, y="class", x="Payload Mass (kg)",
        color="Booster Version Category",
        title='Outcome of Launches from {} with Payload Mass between {} and {} kg'.format
        (entered_site, payload_range[0], payload_range[1]))
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()


C:\Users\nowic\AppData\Local\Temp\ipykernel_23444\2013679643.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\nowic\AppData\Local\Temp\ipykernel_23444\2013679643.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
